In [3]:
import pandas as pd
data = pd.read_csv('../data/processed/data.csv')
data.head()


,race,gender,age,admission_type_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,...,metformin-pioglitazone,change,diabetesMed,discharge_segment,target,numchange,nummeds,diag_1_group,diag_2_group,diag_3_group
0,Caucasian,Female,5,6,1,1,Pediatrics-Endocrinology,41,0,1,...,No,No,No,Otherwise,0,0,0,Diabetes,Other,Other
1,Caucasian,Female,15,1,7,3,NaN,59,0,18,...,No,Ch,Yes,Discharged to home,0,1,1,Other,Diabetes,Other
2,AfricanAmerican,Female,25,1,7,2,NaN,11,5,13,...,No,No,Yes,Discharged to home,0,0,1,Other,Diabetes,Other
3,Caucasian,Male,35,1,7,2,NaN,44,1,16,...,No,Ch,Yes,Discharged to home,0,1,1,Other,Diabetes,Circulatory
4,Caucasian,Male,45,1,7,1,NaN,51,0,8,...,No,Ch,Yes,Discharged to home,0,0,2,Neoplasms,Neoplasms,Diabetes


In [10]:
data.columns

Index(['race', 'gender', 'age', 'admission_type_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'discharge_segment',
       'target', 'numchange', 'nummeds', 'diag_1_group', 'diag_2_group',
       'diag_3_group'],
      dtype='object')

In [4]:
TOP_10_FEATURES = [
    'number_inpatient',      # Rank 1
    'discharge_segment',     # Rank 2
    'diabetesMed',           # Rank 3
    'number_emergency',      # Rank 4
    'metformin',             # Rank 5 (Replaces insulin)
    'diag_1_group',          # Rank 6
    'diag_2_group',          # Rank 10
    'number_diagnoses',      # Rank 9
    'age',                   # Rank 14
    'time_in_hospital'       # Rank 12
]
data_top_10 = data[TOP_10_FEATURES + ['target']]
data_top_10.head()

,number_inpatient,discharge_segment,number_emergency,diag_1_group,diag_2_group,diabetesMed,insulin,number_diagnoses,age,time_in_hospital,target
0,0,Otherwise,0,Diabetes,Other,No,No,1,5,1,0
1,0,Discharged to home,0,Other,Diabetes,Yes,Up,9,15,3,0
2,1,Discharged to home,0,Other,Diabetes,Yes,No,6,25,2,0
3,0,Discharged to home,0,Other,Diabetes,Yes,Up,7,35,2,0
4,0,Discharged to home,0,Neoplasms,Neoplasms,Yes,Steady,5,45,1,0


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Preparar datos
X = data_top_10.drop(columns=['target'])
y = data_top_10['target']

# Calcular scale_pos_weight para manejar el desbalance de clases
# scale_pos_weight = total_negativos / total_positivos
scale_pos_weight = (y == 0).sum() / (y == 1).sum()
print(f"Peso calculado para la clase positiva (scale_pos_weight): {scale_pos_weight:.2f}")

# División estratificada para mantener la proporción de clases
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Pipeline inicial con scale_pos_weight configurado
pipeline_10 = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', XGBClassifier(
        use_label_encoder=False, 
        eval_metric='logloss',
        scale_pos_weight=scale_pos_weight,
        random_state=42
    ))
])

pipeline_10.fit(X_train, y_train)

y_pred = pipeline_10.predict(X_test)
y_pred_proba = pipeline_10.predict_proba(X_test)[:, 1]

print("Reporte de Clasificación (Modelo Base con Balanceo):")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_proba))


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    """
    Función objetivo para optimizar los hiperparámetros de XGBoost dentro de un Pipeline.
    """
    
    # Definir el espacio de búsqueda de hiperparámetros
    param_grid = {
        'xgb__n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'xgb__learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'xgb__max_depth': trial.suggest_int('max_depth', 3, 10),
        'xgb__min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'xgb__gamma': trial.suggest_float('gamma', 0, 5),
        'xgb__subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'xgb__colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'xgb__reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1.0, log=True),
        'xgb__reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1.0, log=True),
        # Permitir que Optuna ajuste ligeramente el peso de la clase si es necesario
        'xgb__scale_pos_weight': trial.suggest_float('scale_pos_weight', scale_pos_weight * 0.8, scale_pos_weight * 1.2)
    }

    # Actualizar los parámetros del Pipeline existente
    pipeline_10.set_params(**param_grid)

    # Evaluación con Validación Cruzada Estratificada (Stratified Cross-Validation)
    # Usamos StratifiedKFold para asegurar que cada fold tenga la misma proporción de clases
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    cv_scores = cross_val_score(
        pipeline_10, 
        X_train, 
        y_train, 
        cv=cv, 
        scoring='roc_auc', 
        n_jobs=-1
    )

    # Retornar el promedio de los scores (Optuna intentará maximizar esto)
    return cv_scores.mean()

# Crear el estudio de Optuna
# Usamos 'maximize' porque queremos maximizar el AUC ROC
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50) # Ejecutar 50 pruebas

print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print("Mejor AUC ROC:", study.best_value)


In [7]:
# Crear el estudio
study = optuna.create_study(direction='maximize', study_name='xgboost_optimization_10')

# Iniciar la optimización (esto puede tardar unos minutos dependiendo de tu hardware)
study.optimize(objective, n_trials=100, show_progress_bar=True)

print("-" * 50)
print("¡Optimización terminada!")
print(f"Mejor AUC ROC obtenido: {study.best_value:.4f}")
print("Mejores Hiperparámetros encontrados:")
print(study.best_params)

[I 2025-11-25 16:11:15,221] A new study created in memory with name: xgboost_optimization_10


  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\asjer\AppData\Local\Temp\ipykernel_17744\2840193074.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'xgb__reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 1.0),
C:\Users\asjer\AppData\Local\Temp\ipykernel_17744\2840193074.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'xgb__reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 1.0)


[I 2025-11-25 16:11:22,432] Trial 0 finished with value: 0.6507244553156821 and parameters: {'n_estimators': 879, 'learning_rate': 0.22106285770405346, 'max_depth': 10, 'min_child_weight': 5, 'gamma': 4.52785307371596, 'subsample': 0.8832758436266293, 'colsample_bytree': 0.5858876742598336, 'reg_alpha': 6.665861031616799e-05, 'reg_lambda': 1.1479940935593975e-05}. Best is trial 0 with value: 0.6507244553156821.


C:\Users\asjer\AppData\Local\Temp\ipykernel_17744\2840193074.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'xgb__reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 1.0),
C:\Users\asjer\AppData\Local\Temp\ipykernel_17744\2840193074.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'xgb__reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 1.0)


...

[I 2025-11-25 16:19:16,397] Trial 97 finished with value: 0.6481007314448083 and parameters: {'n_estimators': 991, 'learning_rate': 0.23989409842662363, 'max_depth': 7, 'min_child_weight': 5, 'gamma': 2.761557892085181, 'subsample': 0.7607038367567444, 'colsample_bytree': 0.5523492457783841, 'reg_alpha': 0.44355923037297, 'reg_lambda': 0.00036491085885108105}. Best is trial 93 with value: 0.653055401003293.


C:\Users\asjer\AppData\Local\Temp\ipykernel_17744\2840193074.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'xgb__reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 1.0),
C:\Users\asjer\AppData\Local\Temp\ipykernel_17744\2840193074.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'xgb__reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 1.0)


[I 2025-11-25 16:19:22,578] Trial 98 finished with value: 0.6529689548182148 and parameters: {'n_estimators': 904, 'learning_rate': 0.014977628215095903, 'max_depth': 8, 'min_child_weight': 4, 'gamma': 2.5906949217542454, 'subsample': 0.7296853913632384, 'colsample_bytree': 0.5109047014998007, 'reg_alpha': 0.30173615591779485, 'reg_lambda': 5.967634037839268e-05}. Best is trial 93 with value: 0.653055401003293.


C:\Users\asjer\AppData\Local\Temp\ipykernel_17744\2840193074.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'xgb__reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 1.0),
C:\Users\asjer\AppData\Local\Temp\ipykernel_17744\2840193074.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'xgb__reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 1.0)


[I 2025-11-25 16:19:28,919] Trial 99 finished with value: 0.6531498893341532 and parameters: {'n_estimators': 902, 'learning_rate': 0.014618414825773144, 'max_depth': 8, 'min_child_weight': 4, 'gamma': 2.6009832922819323, 'subsample': 0.7346229716971907, 'colsample_bytree': 0.5128955327554332, 'reg_alpha': 0.35332029581981156, 'reg_lambda': 5.253447266933403e-05}. Best is trial 99 with value: 0.6531498893341532.
--------------------------------------------------
¡Optimización terminada!
Mejor AUC ROC obtenido: 0.6531
Mejores Hiperparámetros encontrados:
{'n_estimators': 902, 'learning_rate': 0.014618414825773144, 'max_depth': 8, 'min_child_weight': 4, 'gamma': 2.6009832922819323, 'subsample': 0.7346229716971907, 'colsample_bytree': 0.5128955327554332, 'reg_alpha': 0.35332029581981156, 'reg_lambda': 5.253447266933403e-05}


In [8]:
# Obtener los mejores parámetros del estudio
best_params = study.best_params

# Ajustar las claves del diccionario para que coincidan con el Pipeline
# (Añadimos el prefijo 'xgboost__' a cada clave que nos dio Optuna)
pipeline_params = {f'xgb__{k}': v for k, v in best_params.items()}

# 3. Configurar el pipeline con los ganadores
pipeline_10.set_params(**pipeline_params)

# 4. Entrenar modelo definitivo
pipeline_10.fit(X_train, y_train)

# 5. Generar predicciones
# predict() nos da clases (0 o 1)
y_pred = pipeline_10.predict(X_test)
# predict_proba() nos da la probabilidad (0.0 a 1.0), necesaria para el AUC y la curva ROC
y_pred_proba = pipeline_10.predict_proba(X_test)[:, 1]

print("Modelo entrenado y predicciones generadas.")

c:\Users\asjer\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [16:19:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Modelo entrenado y predicciones generadas.


In [9]:

y_pred_proba = pipeline_10.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_proba))


              precision    recall  f1-score   support

           0       0.89      1.00      0.94     18069
           1       0.64      0.01      0.02      2285

    accuracy                           0.89     20354
   macro avg       0.76      0.50      0.48     20354
weighted avg       0.86      0.89      0.84     20354

ROC AUC Score: 0.657798255241608


In [ ]:
# Entrenar el modelo final con los mejores hiperparámetros encontrados
best_params = {f'xgb__{k}': v for k, v in study.best_params.items()}
pipeline_10.set_params(**best_params)
pipeline_10.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred_final = pipeline_10.predict(X_test)
y_pred_proba_final = pipeline_10.predict_proba(X_test)[:, 1]

# Reporte de Clasificación Final
print("Reporte de Clasificación Final (Optimizado):")
print(classification_report(y_test, y_pred_final))
print("AUC ROC Final:", roc_auc_score(y_test, y_pred_proba_final))

# Visualización: Matriz de Confusión
cm = confusion_matrix(y_test, y_pred_final)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.ylabel('Verdadero')
plt.xlabel('Predicho')
plt.show()

# Visualización: Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba_final)
pr_auc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label=f'XGBoost (AUC PR = {pr_auc:.3f})')
plt.xlabel('Recall (Sensibilidad)')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.show()


In [ ]:
import joblib
import os

# Crear directorio models si no existe
if not os.path.exists('../models'):
    os.makedirs('../models')

# Guardar el mejor modelo
model_path = '../models/xgboost_optimized.pkl'
joblib.dump(pipeline_10, model_path)
print(f"Modelo optimizado guardado en {model_path}")
